# Flask (продолжение)

## Flask и Базы данных

Для того, чтобы удобно было работать с базами в обычных программах (особенно в приложениях), придумали **ORM** (Object-Relational Mapping, объектно-реляционное отображение, или преобразование).

ORM $-$ это специальный инструмент для перевода объектов в БД в удобную форму для этого языка программирования (например, в классы питона). Так как мы пока не вспоминали классы глубоко, можно привести пример `pymorphy`, где разбор имел разные атрибуты (которые вызывались через точку) и внутри них значения или более сложная структура (`.tag` имел еще `.tag.POS` и т.д.).

Предположим, что мы хотим сделать лингвистическую анкету и у нас есть такая вводная информация:


- Мы хотим собрать информацию про информанта (гендер, уровень образования и возраст)
- У нас есть два вопроса на оценку предложений
- И нужно куда-то сохранять результаты


Проще всего все положить в БД и потом просто работать с этим файлом.

*Таблица про информантов*:

| User      |
|:----------|
| id_user   |
| gender    |
| education |
| age       |

*Таблица про вопросы*:

| Questions |
|:----------|
| id_q      |
| text      |

*Таблица про ответы информантов на вопросы*:

| Answers   |
|:----------|
| id_user   |
| ans_q1    |
| ans_q2    |

In [ ]:
import sqlite3

db = sqlite3.connect('test.db')
cur = db.cursor()

In [ ]:
cur.execute(
    """CREATE TABLE answers (
    id_user INTEGER PRIMARY KEY AUTOINCREMENT,
    ans_q1` INTEGER,
    ans_q2` INTEGER )
    """)

cur.execute(
    """CREATE TABLE questions (
    id_q INTEGER PRIMARY KEY AUTOINCREMENT,
    text TEXT
    )""")

cur.execute(
    """CREATE TABLE
    user (
    id_user INTEGER PRIMARY KEY AUTOINCREMENT,
    gender TEXT,
    education TEXT,
    age INTEGER )""")

db.commit()

### Как подключить базу к приложению?

Мы должны импортировать фласк и использовать еще один специальный модуль [`flask-sqlalchemy`](https://flask-sqlalchemy.readthedocs.io/en/stable/), который позволяет работать с базой. Главное $-$ это прописать путь к базе. Там есть часть `sqlite3:///` $-$ это обозначение для того, что мы работает с таким типом базы, а дальше путь внутри проекта.

In [ ]:
# !pip install -U Flask-SQLAlchemy --q

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
db = SQLAlchemy(app)

### Как передать устройство своей базы?

Нам необходимо объяснить питону, где какие типы данных у нас лежат, для этого нам нужно описать каждую таблицу.

Не будем пока углубляться в классы питона, просто делаем по образцу:
1. Каждая таблица описывается отдельно (класс, наследующий от `db.Model`):
    - с помощью `__tablename__` передается ее имя
    - названия столбиков `db.Column` совпадают с названиями в нашей таблице
2. Используется специальная "обертка" для столбца, где можно прописать, какой это тип данных (при помощи `db.Integer`, `db.Text` и т.д.) и если это первичный ключ, то на это указать (`primary_key=True`).

In [ ]:
class User(db.Model):
    __tablename__ = 'user'  # имя таблицы
    id_user = db.Column(db.Integer, primary_key=True) # имя колонки = специальный тип (тип данных, первичный ключ)
    gender = db.Column(db.Text)
    education = db.Column(db.Text)
    age = db.Column(db.Integer)


class Questions(db.Model):
    __tablename__ = 'questions'
    id_q = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.Text)


class Answers(db.Model):
    __tablename__ = 'answers'
    id_user = db.Column(db.Integer, primary_key=True)
    ans_q1 = db.Column(db.Integer)
    ans_q2 = db.Column(db.Integer)

После того, как вы определяете все модели и таблицы нужно их создать при помощи `create_all()`. Данное действие (как и любое другое вне запроса) требует `app_context()`:

In [ ]:
with app.app_context():
    db.create_all()

Вручную добавим несколько вопросов в анкету при помощи `add(object)` / `add_all(list)`:

In [ ]:
q1 = Questions(
    id_q=1,
    text='Как вы относитесь к пицце с ананасами?'
    )
q2 = Questions(
    id_q=2,
    text='Как вы относитесь к Санкт-Петербургу?'
    )

with app.app_context():
    db.session.add_all([q1, q2])
    db.session.commit()

### Как читать из базы (простые вещи)?

Простейшие запросы можно делать прямо к `User`, `Questions`, `Answers`. Например, мы хотим на нашу страницу с анкетой предложения для оценки брать прямо из базы.

Мы должны:
1. Получить эти вопросы
2. Передать на страницу
3. Пройти по вопросам и сгенерировать анкету

Создадим путь `/questions` и там считаем вопросы из базы и передадим на страницу через `render_template`.

In [ ]:
from flask import render_template

In [ ]:
@app.route('/questions')
def question_page():
    # имя_таблицы.query.взять_все()
    questions = Questions.query.all()
    return render_template(
        'questions.html',
        questions=questions
    )

А как теперь это использовать, говорили же про атрибуты? Где они?

Предположим, что у нас есть N вопросов (2 в нашем случае) и они одинаково устроены. Мы можем просто сгенерировать места для ответов и они все по очереди будут выводиться.

- `{{question.text}}` $-$ выводит нам поле `text` из вопроса.
- `name="q{{ question.id_q }}"` $-$ использует id вопроса, чтобы получить имена для элементов например, `name="q1"` (чтобы мы могли потом достать ответ именно на первый вопрос).

Лежит в [`questions.html`](https://github.com/hse-ling-python/seminars/blob/master/flask_applications/flask_db_queries_2024_data/templates/questions.html).

```html
{% for question in questions %}
    <div class="row">
        <p class="col-md-6">{{question.text}}</p>
        <table class="col-md-8">
            <tr>
                <td><input class="radio" type="radio" name="q{{ question.id_q }}" value=5></td>
                <td><input class="radio" type="radio" name="q{{ question.id_q }}" value=4></td>
                <td><input class="radio" type="radio" name="q{{ question.id_q }}" value=3></td>
                <td><input class="radio" type="radio" name="q{{ question.id_q }}" value=2></td>
                <td><input class="radio" type="radio" name="q{{ question.id_q }}" value=1></td>
            </tr>
            <tr>
                <td>отлично</td>
                <td>хорошо</td>
                <td>норм</td>
                <td>плохо</td>
                <td>ужасно</td>
            </tr>
        </table>
    </div>
{% endfor %}
```



### Как писать в базу?

Мы указали в форме ```<form action="/process">```, чтобы данные отправлялись на путь process и мы там будем их обрабатывать:


1. Достать все параметры из адреса с GET параметрами (типа ```gender=female&education=hse```)
2. Записать в базу
3. Сохранить
4. Сказать пользователю, что все ок
5. Если пришел пустой запрос, то отправить проходить анкету

In [ ]:
from flask import request, redirect, url_for

In [ ]:
@app.route('/process', methods=['get'])
def answer_process():

    # если пустой запрос, то отправить проходить анкету
    if not request.args:
        return redirect(url_for('question_page'))

    # получаем значения ответов
    gender = request.args.get('gender')
    education = request.args.get('education')
    age = request.args.get('age')

    # записываем в базу
    user = User(
        age=age,
        gender=gender,
        education=education
    )
    db.session.add(user)
    db.session.commit()

    # обновляем user'a, чтобы его ответ записать с таким же id
    db.session.refresh(user)

    # это же делаем с ответом
    q1 = request.args.get('q1')
    q2 = request.args.get('q2')
    answer = Answers(
        id_user=user.id_user,
        ans_q1=q1,
        ans_q2=q2
    )
    db.session.add(answer)
    db.session.commit()

    # пользователь попадает на страницу, где напсано только Ок
    return 'Ok'

### Как получить из базы что-то сложное?

Нам нужна какая-то статистика на сайт, чтобы любопытные могли посмотреть сколько человек уже прошли анкету и какие примерно ответы получаются. Чтобы использовать функции типа среднее, максимум, минимум и т.д. нам нужен `sqlalchemy.func`.

In [ ]:
from sqlalchemy import func

In [ ]:
@app.route('/stats')
def stats():
    # заводим словарь для значений (чтобы не передавать каждое в render_template)
    all_info = {}

    age_stats = db.session.query(
        func.avg(User.age), # средний возраст AVG(user.age)
        func.min(User.age), # минимальный возраст MIN(user.age)
        func.max(User.age)  # максимальный возраст MAX(user.age)
    ).one() # берем один результат (он всего и будет один)

    all_info['age_mean'], all_info['age_min'], all_info['age_max'] = age_stats

    # это простой запрос, можно прямо у таблицы спросить
    all_info['total_count'] = User.query.count() # SELECT COUNT(age) FROM user

    # SELECT AVG(q1) FROM answers
    all_info['q1_mean'] = db.session.query(func.avg(Answers.ans_q1)).one()[0]

    # SELECT q1 FROM answers
    q1_answers = db.session.query(Answers.ans_q1).all()

    return render_template('results.html', all_info=all_info)

In [ ]:
if __name__ == '__main__':
    app.run()

**Остается адаптировать под себя и поработать над оформлением!**

## Шаблоны страниц

Чтобы сайт проще было конструировать, применяются шаблоны. Мы уже смотрели на Jinja templates и как можно подставлять значения переменных в страницы.

Можно соединять разные блоки в странице, чтобы не копировать части HTML-кода, который повторяется на всех или многих страницах. Например, в ВК есть стандартная панель сбоку, которая не меняется от страницы к странице и меняется только центральная полоса. Или на сайтах бывает боковое меню, верхнее меню, новостная лента, которые постоянны в шаблоне, а меняется только часть страницы.

Для этого создается базовый шаблон страницы (`base.html`), который потом просто дополняется информацией, которая отличается на страницах. Мы можем на ней загрузить наши стили, скрипты и прочие зависимости и не думать про это позже.

Поместим сюда меню, так как оно не меняется, стили, добавим поля основной части страницы.

Мы также указываем блоки, куда потом вставим наши отличающиеся кусочки.

Для этого пропишем `{% block title %} {% endblock %}` и `{% block content %} {% endblock %}`.

`title` и `content` $-$ это названия наших блоков, их содержимое пока пустое, мы его заменим.

Так выглядит [`base.html`](https://github.com/hse-ling-python/seminars/blob/master/flask_applications/imdb_site/templates/base.html):

```html
<!DOCTYPE html>
<html lang="en">
    
<head>
    <meta charset="UTF-8">
    <title>{% block title %} {% endblock %}</title>
         <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS, JS-->
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" crossorigin="anonymous">
    <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js" integrity="sha384-DfXdz2htPH0lsSSs5nCTpuj/zy4C+OGpamoFVy38MVBnE+IbbVYUew+OrCXaRkfj" crossorigin="anonymous"></script>
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-ho+j7jyWK8fNQe+A12Hb8AhRq26LrZ/JpcUGGOn+Y7RsweNrtN/tE3MoK7ZeZDyx" crossorigin="anonymous"></script>
</head>
    
<body>
    <!-- вот отсюда https://getbootstrap.com/docs/4.5/components/navbar/ -->
    <nav class="navbar navbar-expand-lg navbar-light bg-light">
      <a class="navbar-brand" href="#">База фильмов</a>
      <button class="navbar-toggler" type="button" data-toggle="collapse" data-target="#navbarNavAltMarkup" aria-controls="navbarNavAltMarkup" aria-expanded="false" aria-label="Toggle navigation">
        <span class="navbar-toggler-icon"></span>
      </button>
      <div class="collapse navbar-collapse" id="navbarNavAltMarkup">
        <div class="navbar-nav">
          <a class="nav-link" href="/rating">Рейтинг фильмов</a>
          <a class="nav-link" href="/search">Поиск</a>
        </div>
      </div>
    </nav>

    <div style="margin: 50px 10%">
        {% block content %} {% endblock %}
    </div>


</body>
</html>
```

Теперь на главной странице просто подставим информацию.

Указываем, что мы этим кодом раширяем `base.html`. Вписываем блоки

Так выглядит [`index.html`]():

```html
{% extends 'base.html' %}

{% block title %} База фильмов {% endblock %}

{% block content %}
    <h1>Текст главной страницы</h1>
{% endblock %}
```

## А как работать с чем-то готовым?

Попробуем подключить базу данных к нашему приложению и сделать [сайт про фильмы](https://github.com/hse-ling-python/seminars/tree/master/flask_applications/imdb_site).

На самом деле, обычно для описания таблиц создается отдельный файл, например [`models.py`](https://github.com/hse-ling-python/seminars/blob/master/flask_applications/imdb_site/models.py).

In [ ]:
# models.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

# дальше описания таблиц

А в файле приложения мы импортируем этот объект и подключаем к приложению.

In [ ]:
# app.py

# создаем приложение
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///imdb_small_indexed.db'
# чтобы ок работать с изменениями в базе (коммиты и обновления информации, если такие есть)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# указываем базе, какое наше приложение
db.app = app
db.init_app(app)

Теперь можно пользоваться.

Вернемся к нашим моделям (таблицам).

In [ ]:
# models.py

class Type(db.Model):

    __tablename__ = "film_types"

    type_id = db.Column('id', db.Integer, primary_key=True)
    name = db.Column('film_type', db.Text)

Можно указать, что столбец ссылается на ключ из другой таблицы, например, как у нас в `crew`.

**ForeignKey**

In [ ]:
from flask_sqlalchemy import ForeignKey, PrimaryKeyConstraint

In [ ]:
# models.py

class Crew(db.Model):
    __tablename__ = 'crew'
    __table_args__ = (PrimaryKeyConstraint('title_id', 'person_id'),)

    film_id = db.Column('title_id', db.Integer, ForeignKey('titles.title_id'))
    person_id = db.Column('person_id', db.Integer, ForeignKey('people.person_id'))

Мы можем сделать так, чтобы к фильму привязались люди, которые там участвуют (в атрибуте будет лежать список)

Для этого мы используем **`relationship`**. Так как мы здесь соединяем две таблицы через вспомогательную, то мы это делаем, указав, с каким объектом мы соединяем и какая вспомогательная таблица.

Для рейтинга мы просто связываем две таблицы, где связь один-один, мы можем это сделать сразу, в `primaryjoin` указываем, как мы свзяываем таблицы, то есть какие столбцы соответствуют друг другу.

In [ ]:
# models.py

class Film(db.Model):
    __tablename__ = "titles"

    # имя колонки = специальный тип (тип данных, первичный ключ)
    film_id = db.Column('title_id', db.Integer, primary_key=True)

    rating = db.relationship('Rating', uselist=False, primaryjoin="Film.film_id==Rating.film_id")

    ...

    crew = db.relationship("Person", secondary='crew') # с кем, через какую таблицу

class Person(db.Model):
    __tablename__ = "people"

    person_id = db.Column('person_id', db.Integer, primary_key=True)
    name = db.Column('name', db.Text)
    born = db.Column('born', db.Integer)
    died = db.Column('died', db.Integer)

    films = db.relationship("Film", secondary='crew') # с кем, через какую таблицу

## Запросы к базе и использование объектов

Можно импортировать наши модели (классы) и использовать их для запросов к базе.

### Запрос по ключу

Самый простой пример - это запрос одного объекта по айди. Сделаем страницу для человека, URL будет выгялдеть как `http://127.0.0.1:5000/person/186505`. Мы получаем айди человека и просто в базе получаем информацию о человеке с таким ключом.

Используем метод `get` для получения по ключу. Обратно нам выдастся объект типа `Person`, мы можем использовать атрибуты, которые мы прописали в модели. Передаем этот объект на страницу и вписываем в шаблон переменные.

In [ ]:
@app.route("/person/<person_id>")
def person_page(person_id):
    person = Person.query.get(person_id)
    return render_template("person.html", person=person)

Тем временем в [`person.html`](https://github.com/hse-ling-python/seminars/blob/master/flask_applications/imdb_site/templates/person.html)

```html
{% extends 'base.html' %}

{% block title %} {{ person.name }}{% endblock %}

{% block content %}
    <table class="table">
        <tr>
            <td>Имя</td>
            <td>{{ person.name }}</td>
        </tr>
        <tr>
            <td>Год рождения</td>
            <td>{{ person.born }}</td>
        </tr>
        {% if person.died %}
        <tr>
            <td>Год смерти</td>
            <td>{{ person.died }}</td>
        </tr>
        {% endif %}
        <tr>
            <td>Фильмы</td>
            <td>
                {% for film in person.films %}
                    {{ film.title }} <br>
                {% endfor %}
            </td>
        </tr>
    </table>
{% endblock %}

```

### Топ фильмов

В запросах от класса мы не можем нормально искать по атрибутам, которые привязаны (например, список актеров к фильму), но мы можем сделать `join`.

Мы делаем запрос через `db.session.query`, в нашем случае это будет что-то вроде курсора. Мы запрашиваем таблицу `Film`, соединяем ее с `Rating`.

Теперь фильтруем. Добавляем фильтр, что число оценок больше 100 тысяч, сортируем по рейтингу по убыванию.

Берем топ-250. Получаем ответ с помощью `all` (как `fetchall`).

Мы получили список объектов типа `Film`, которые отсортированы и отфильтрованы как мы просили. Это удобнее, чем просто кортежи, где нам надо помнить, что на какой позиции лежит, мы просто тут можем обращаться по именам атрибутов.

In [ ]:
# app.py

@app.route("/rating")
def rating():
    rating = db.session.query(Film)\
        .join(Rating)\
        .filter(Rating.votes > 100000)\
        .order_by(-Rating.value)\
        .limit(250).all()
    return render_template("rating.html", rating=rating)

В шаблоне страницы мы можем идти по объектам (как в обычном списке в питоне), но мы хотим помнить номер порядковый, поэтому попробуем идти по индексам, для это сделаем range от 0 до длины списка).

```html
{% extends 'base.html' %}

{% block title %} Рейтинг фильмов {% endblock %}

{% block content %}
    <table class="table">
        {% for i in range(0, rating|length) %}
            <tr>
                <td scope="row">{{ i + 1 }}</td>
                <td>{{ rating[i].rating.value }}</td>
                <td>{{ rating[i].premiered }}{% if rating[i].ended %}-{{ rating[i].ended }}{% endif %}</td>
                <td><a href="/film/{{ rating[i].film_id }}" target="_blank">{{ rating[i].title }}</a></td>
            </tr>
        {% endfor %}
    </table>
{% endblock %}
```

## PythonAnyWhere (PAW)

Теперь посмотрим, как выложить наш сайт на хостинг так, чтобы им могли пользоваться другие. Для этого:
1. Идем на сайт https://www.pythonanywhere.com/ и регистрируемся там
2. Загружаем все файлы в отдельную папку внутри [вкладки файлы](https://www.pythonanywhere.com/user/tokubetsu/files), там надо перейти в папку `home/USERNAME`, создать там папку проекта и работать в ней. Файлы переносить можно руками, можно через гитхаб (есть [вот такая инструкция](https://help.pythonanywhere.com/pages/UploadingAndDownloadingFiles))
3. Проверьте, что все настройки сайта в файлах актуальны (пути, хостинг и прочее, что могло поменятся при переезде с локального сервера)
4. Теперь будем настраивать окружение (версию питона и все библиотеки). Идем во вкладку `Consoles`, создаем там новую Bash консоль и создаем нужное окружение с версией питона (скорее всего вам подойдет python=3.10):
    
    `mkvirtualenv --python=/usr/bin/python3.10 VENV_NAME`

    Вот так можно активировать уже созданное окружение:
        
    `workon VENV_NAME`

5. Устанавливаем все нужные библиотеки (в том числе Flask)
6. Идем во вкладку `Web` и создаем там новое приложение.


[Более подробная инструкция](https://help.pythonanywhere.com/pages/Flask/)
